In [24]:
import sys
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from collections import Counter
#import advertools as adv
from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
#from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
from scipy.special import logsumexp

In [12]:
RD = pd.read_csv('raw_data_recipe.csv')

In [13]:
print(RD.shape)

(49698, 9)


In [155]:
print(RD.columns)

Index(['recipe_id', 'recipe_name', 'aver_rate', 'image_url', 'review_nums',
       'ingredients', 'cooking_directions', 'nutritions', 'reviews'],
      dtype='object')


In [156]:
print(RD.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49698 entries, 0 to 49697
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   recipe_id           49698 non-null  int64  
 1   recipe_name         49698 non-null  object 
 2   aver_rate           49698 non-null  float64
 3   image_url           49698 non-null  object 
 4   review_nums         49698 non-null  int64  
 5   ingredients         49698 non-null  object 
 6   cooking_directions  49698 non-null  object 
 7   nutritions          49698 non-null  object 
 8   reviews             49698 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 3.4+ MB
None


In [180]:
IG = RD['ingredients']

In [159]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [160]:
 from nltk.tokenize import sent_tokenize, word_tokenize

In [172]:
RD['ingredients'] = RD['ingredients'].astype('str') 

In [181]:
IG = RD['ingredients']

In [163]:
from nltk import PorterStemmer

In [164]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)

In [165]:
for col in RD.columns:
    print(col, RD[col].isnull().sum())

recipe_id 0
recipe_name 0
aver_rate 0
image_url 0
review_nums 0
ingredients 0
cooking_directions 0
nutritions 0
reviews 0


In [199]:
RD['ingredients'] = [''.join(map(str, l)) for l in RD['ingredients']]

In [ ]:
RD['ingredients1'] = RD['ingredients'].replace('^',' ')

In [ ]:
print(RD)

In [19]:
print(RD.columns)

Index(['recipe_id', 'recipe_name', 'aver_rate', 'image_url', 'review_nums',
       'ingredients', 'cooking_directions', 'nutritions', 'reviews'],
      dtype='object')


In [25]:
def convert_lower_case(data):
    return np.char.lower(data)

In [26]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [27]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [28]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [29]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    
    new_text = ""
    for w in tokens:

        for character in w:
            if character.isdigit():
                w = re.sub("[A-Za-z]+", lambda ele: " " + ele[0] + " ", w)
        
        try:
            
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
        
    new_text = np.char.replace(new_text, "-", " ")
    
    return new_text

In [30]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~"
    
    data = np.char.replace(data, " m\\n", ' minutes ' )
    data = np.char.replace(data, 'h\\', ' hours ')
    data = np.char.replace(data, ' h ', ' hours ')

    data = np.char.replace(data, "\\n", ' ')

    for i in range(len(symbols)):
        
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
        
    data = np.char.replace(data, ',', '')
    data = np.char.replace(data, ' f ', ' fahrenheit ')
    data = np.char.replace(data, ' c ', ' celcius ')
    data = np.char.replace(data, " u'", ' ' )

    return data

In [31]:
def remove_otherwords(data):
    data = np.char.replace(data, 'prep', ' ')
    data = np.char.replace(data, 'directions', ' ')
    return data

In [32]:
def preprocess(data):
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) 
    data = convert_numbers(data)
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = remove_stop_words(data) 
    return data

In [33]:
processed_text = []
for text in RD['cooking_directions']:
    processed_text.append(word_tokenize(preprocess(text)))

In [37]:
print(preprocess(RD['cooking_directions'][0]))

 directions prep ncook hours 45 minutes ready 11 hours 50 minutes preheat oven 200 degrees fahrenheit 95 degrees celcius season pork belly paprika salt pepper tightly wrap pork twice heavy duty aluminum foil place baking sheet bake preheated oven hours turn oven let pork rest oven hour remove meat oven leaving wrapped aluminum foil refrigerate least hours overnight remove pork foil slice across grain inch thick slices working batches cook pork non stick skillet medium heat golden crisped minutes per slice


In [38]:
processed_text = []
for text in RD['ingredients']:
    processed_text.append(word_tokenize(preprocess(text)))

In [39]:
print(preprocess(RD['ingredients'][0]))

 pork belly smoked paprika kosher salt ground black pepper


In [41]:
print(preprocess(RD['cooking_directions'][0]))
print(preprocess(RD['ingredients'][0]))

 directions prep ncook hours 45 minutes ready 11 hours 50 minutes preheat oven 200 degrees fahrenheit 95 degrees celcius season pork belly paprika salt pepper tightly wrap pork twice heavy duty aluminum foil place baking sheet bake preheated oven hours turn oven let pork rest oven hour remove meat oven leaving wrapped aluminum foil refrigerate least hours overnight remove pork foil slice across grain inch thick slices working batches cook pork non stick skillet medium heat golden crisped minutes per slice
 pork belly smoked paprika kosher salt ground black pepper


In [46]:
print(preprocess(RD['cooking_directions'][0]))

 pork belly smoked paprika kosher salt ground black pepper directions prep ncook hours 45 minutes ready 11 hours 50 minutes preheat oven 200 degrees fahrenheit 95 degrees celcius season pork belly paprika salt pepper tightly wrap pork twice heavy duty aluminum foil place baking sheet bake preheated oven hours turn oven let pork rest oven hour remove meat oven leaving wrapped aluminum foil refrigerate least hours overnight remove pork foil slice across grain inch thick slices working batches cook pork non stick skillet medium heat golden crisped minutes per slice


In [48]:
RD["Total"] = ""
RD.to_csv("raw_data_recipe.csv", index=False)

In [50]:
RD["Total"] =RD["recipe_name"] + RD["ingredients"] + RD["cooking_directions"]

In [51]:
print(preprocess(RD['Total'][0]))

 homemade baconpork belly smoked paprika kosher salt ground black pepperpork belly smoked paprika kosher salt ground black pepper directions prep ncook hours 45 minutes ready 11 hours 50 minutes preheat oven 200 degrees fahrenheit 95 degrees celcius season pork belly paprika salt pepper tightly wrap pork twice heavy duty aluminum foil place baking sheet bake preheated oven hours turn oven let pork rest oven hour remove meat oven leaving wrapped aluminum foil refrigerate least hours overnight remove pork foil slice across grain inch thick slices working batches cook pork non stick skillet medium heat golden crisped minutes per slice
